<a href="https://colab.research.google.com/github/afifrizkyandika11551100310/SENTIMENT_ANALYSIS_FULL_DEEP_NN/blob/main/Ruled_based_polarity_and_Deep_Learning_sentiment_classification_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Importing Packages

In [ ]:
import pandas as pd
import numpy as np
import re
import plotly.graph_objects as go
import pandas as pd
import nltk
import textblob
import ast
import plotly.express as px
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
nltk.download('vader_lexicon')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from keras import Sequential
from keras import models
from keras import layers
from keras import regularizers
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Flatten, Bidirectional, Dropout, Conv1D, GlobalMaxPooling1D, GRU, MaxPooling1D
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from nltk.sentiment.vader import SentimentIntensityAnalyzer as vad

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
pip install pyspellchecker

     |████████████████████████████████| 2.7 MB 5.4 MB/s 


## 2. Importing Files

In [ ]:
pd.options.display.max_colwidth = -1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [ ]:
data = pd.read_csv('spam.csv', usecols=['Message'])
data.head(10)

,Message
0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,Ok lar... Joking wif u oni...
2,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,U dun say so early hor... U c already then say...
4,"Nah I don't think he goes to usf, he lives around here though"
5,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"
6,Even my brother is not like to speak with me. They treat me like aids patent.
7,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
8,WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
9,Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030


## 3. Data Cleaning

In [ ]:
data['Message'] = data['Message'].str.lower()

In [ ]:
#Cleansing

#Fungsi untuk membersihkan data
def Cleansing(text):
    #Menghapus tab, new line dan back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    #Menghapus emoticon dan non ASCII
    text = text.encode('ascii', 'replace').decode('ascii')
    #Menghapus mention, hashtag dan link
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    #Menghapus URL
    return text.replace("http://", " ").replace("https://", " ")

#Menerapkan fungsi untuk membersihkan data
data['Message'] = data['Message'].apply(Cleansing)

#Fungsi untuk menghapus angka
def Hapus_Angka(text):
    return  re.sub(r"\d+", "", text)

#Menerapkan fungsi untuk menghapus angka
data['Message'] = data['Message'].apply(Hapus_Angka)

#Fungsi untuk menghapus huruf tunggal
def Hapus_HurufTunggal(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

#Menerapkan fungsi untuk menghapus huruf tunggal
data['Message'] = data['Message'].apply(Hapus_HurufTunggal)

#Menampilkan hasil cleansing
print('Hasil Cleansing : \n')
data

Hasil Cleansing : 



,Message
0,"go until jurong point, crazy.. available only in bugis great world la buffet... cine there got amore wat..."
1,ok lar... joking wif oni...
2,free entry in wkly comp to win fa cup final tkts st may . text fa to to receive entry question(std txt rate)&' apply over'
3,dun say so early hor... already then say...
4,"nah don' think he goes to usf, he lives around here though"
...,...
5567,"this is the nd time we have tried contact . have won the ? pound prize. claim is easy, call now! only per minute. bt-national-rate."
5568,will ? going to esplanade fr home?
5569,"pity, * was in mood for that. so...any other suggestions?"
5570,the guy did some bitching but acted like ' be interested in buying something else next week and he gave it to us for free


In [ ]:
# hilangkan link
data['Message']=data['Message'].apply(lambda x:re.sub(r"http\S+", "", x))
# hilangkan karakter spesial 
data['Message']=data['Message'].apply(lambda x:' '.join(re.findall(r'\w+', x)))
# hilangkan karakter single
data['Message']=data['Message'].apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))
# hilangkan nomor
data['Message']=data['Message'].apply(lambda x:re.sub(r'[0-9]', '', x))

In [ ]:
data=data[['Message']]
data.head(10)

,Message
0,go until jurong point crazy available only in bugis great world la buffet cine there got amore wat
1,ok lar joking wif oni
2,free entry in wkly comp to win fa cup final tkts st may text fa to to receive entry question std txt rate apply over
3,dun say so early hor already then say
4,nah don think he goes to usf he lives around here though
5,freemsg hey there darling it been week now and no word back like some fun you up for it still tb ok xxx std chgs to send to rcv
6,even my brother is not like to speak with me they treat me like aids patent
7,as per your request melle melle oru minnaminunginte nurungu vettam has been set as your callertune for all callers press to copy your friends callertune
8,winner as valued network customer you have been selected to receivea prize reward to claim call claim code kl valid hours only
9,had your mobile months or more entitled to update to the latest colour mobiles with camera for free call the mobile update co free on


In [ ]:
from collections import Counter
cnt = Counter()
for text in data["Message"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('to', 2250),
 ('you', 2245),
 ('the', 1337),
 ('and', 980),
 ('in', 904),
 ('is', 896),
 ('me', 807),
 ('my', 767),
 ('it', 752),
 ('for', 711)]

In [ ]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

data["Message"] = data["Message"].apply(lambda text: remove_freqwords(text))
data.head()

,Message
0,go until jurong point crazy available only bugis great world la buffet cine there got amore wat
1,ok lar joking wif oni
2,free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate apply over
3,dun say so early hor already then say
4,nah don think he goes usf he lives around here though


In [ ]:
n_rare_words = 100
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

data["Message"] = data["Message"].apply(lambda text: remove_rarewords(text))
data.head()

,Message
0,go until jurong point crazy available only bugis great world la buffet cine there got amore wat
1,ok lar joking wif oni
2,free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate apply over
3,dun say so early hor already then say
4,nah don think he goes usf he lives around here though


In [ ]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [ ]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

data["Message"] = data["Message"].apply(lambda text: remove_stopwords(text))
data.head()

,Message
0,go jurong point crazy available bugis great world la buffet cine got amore wat
1,ok lar joking wif oni
2,free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate apply
3,dun say early hor already say
4,nah think goes usf lives around though


## 4. Language Preprocessing 

### 4.1 Lemmatisasi

In [ ]:
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
# lemmatisasi
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}
def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

data["text_terlemmatisasi"] = data["Message"].apply(lambda text: lemmatize_words(text))
data.head()

,Message,text_terlemmatisasi
0,go jurong point crazy available bugis great world la buffet cine got amore wat,go jurong point crazy available bugis great world la buffet cine get amore wat
1,ok lar joking wif oni,ok lar joking wif oni
2,free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate apply,free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate apply
3,dun say early hor already say,dun say early hor already say
4,nah think goes usf lives around though,nah think go usf life around though


In [ ]:
from spellchecker import SpellChecker
spell = SpellChecker(distance=1)
def Correct(x):
  return spell.correction(x)
data['spell_correction'] = data['text_terlemmatisasi'].apply(Correct)
data.head()

,Message,text_terlemmatisasi,spell_correction
0,go jurong point crazy available bugis great world la buffet cine got amore wat,go jurong point crazy available bugis great world la buffet cine get amore wat,go jurong point crazy available bugis great world la buffet cine get amore wat
1,ok lar joking wif oni,ok lar joking wif oni,ok lar joking wif oni
2,free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate apply,free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate apply,free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate apply
3,dun say early hor already say,dun say early hor already say,dun say early hor already say
4,nah think goes usf lives around though,nah think go usf life around though,nah think go usf life around though


In [ ]:
new_data = data.copy()
new_data.drop(columns = ['Message', 'text_terlemmatisasi'], inplace = True)
new_data.head()

,spell_correction
0,go jurong point crazy available bugis great world la buffet cine get amore wat
1,ok lar joking wif oni
2,free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate apply
3,dun say early hor already say
4,nah think go usf life around though


## 5. Text Polarity

In [ ]:
sentiment=[]
subjectivity=[]
sentiment_score=[]
for i in new_data['spell_correction'].values:
    Tweet = textblob.TextBlob(i)
    sentiment_score.append(Tweet.sentiment[0])
    if Tweet.sentiment[0]<0:
        sentiment.append('negatif')
    elif Tweet.sentiment[0]>0:
        sentiment.append('positif')
    else:
        sentiment.append('netral')
    subjectivity.append(Tweet.sentiment[1])

In [ ]:
new_data['Sentiment Score']=sentiment_score
new_data['Subjectivity']=subjectivity
new_data['Overall_Sentiment']=sentiment

In [ ]:
new_data.head(10)

,spell_correction,Sentiment Score,Subjectivity,Overall_Sentiment
0,go jurong point crazy available bugis great world la buffet cine get amore wat,0.200000,0.683333,positif
1,ok lar joking wif oni,0.500000,0.500000,positif
2,free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate apply,0.400000,0.733333,positif
3,dun say early hor already say,0.100000,0.300000,positif
4,nah think go usf life around though,0.000000,0.000000,netral
5,freemsg hey darling week word back like fun still tb ok xxx std chgs send rcv,0.266667,0.233333,positif
6,even brother like speak treat like aid patent,0.000000,0.000000,netral
7,per request melle melle oru minnaminunginte nurungu vettam set callertune caller press copy friend callertune,0.000000,0.000000,netral
8,winner value network customer select receivea prize reward claim call claim code kl valid hour,0.000000,0.000000,netral
9,mobile month entitle update late colour mobile camera free call mobile update co free,0.166667,0.733333,positif


In [ ]:
new_data.to_csv('polarity_dataset.csv', index = False, header = True)

## 6. Klasifikasi Sentiment ML

In [ ]:
new_data['Overall_Sentiment'] = new_data['Overall_Sentiment'].replace(to_replace = "positif", value = 2)
new_data['Overall_Sentiment'] = new_data['Overall_Sentiment'].replace(to_replace = "negatif", value = 0)
new_data['Overall_Sentiment'] = new_data['Overall_Sentiment'].replace(to_replace = "netral", value = 1)

In [ ]:
X = new_data['spell_correction'].astype(str)
y = new_data['Overall_Sentiment']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
X_train

301     awesome remember last time get somebody high first time diesel                                                                 
3341    like make throw smoke friend car one time awesome                                                                              
4870    tension face smile face waste face innocent face terror face cruel face romantic face lovable face decent face lt gt joker face
932     congratulation ore mo owo wa enjoy wish many happy moment fro wherever go                                                      
2941    new message please call                                                                                                        
                 ...                                                                                                                   
2814    enjoy shower possessiveness pour ur loved one bcoz world lie golden gift love truly                                            
112     go dinner msg                           

In [ ]:
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=500000)
vectoriser.fit(X_train)
print(f'Vectoriser fitted.')
print('No. of feature_words: ', len(vectoriser.get_feature_names()))

Vectoriser fitted.
No. of feature_words:  30237


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
vector_X_train = vectoriser.transform(X_train)
vector_X_test  = vectoriser.transform(X_test)

In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix, classification_report

BNBmodel = BernoulliNB(alpha = 1)
BNBmodel.fit(vector_X_train, y_train)
pred_1 = BNBmodel.predict(vector_X_test)
print(classification_report(y_test, pred_1)) 

              precision    recall  f1-score   support

           0       1.00      0.04      0.08       149
           1       0.56      0.98      0.71       477
           2       0.89      0.49      0.63       489

    accuracy                           0.64      1115
   macro avg       0.82      0.50      0.47      1115
weighted avg       0.76      0.64      0.59      1115



In [ ]:
from xgboost import XGBClassifier

model_xgb = XGBClassifier()
model_xgb.fit(vector_X_train, y_train)
pred_2 = model_xgb.predict(vector_X_test)
print(classification_report(y_test, pred_2)) 

              precision    recall  f1-score   support

           0       0.88      0.33      0.48       149
           1       0.69      0.99      0.81       477
           2       0.96      0.74      0.84       489

    accuracy                           0.79      1115
   macro avg       0.84      0.69      0.71      1115
weighted avg       0.83      0.79      0.78      1115



In [ ]:
from sklearn.linear_model import LogisticRegression

LRmodel = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1)
LRmodel.fit(vector_X_train, y_train)
pred_3 = LRmodel.predict(vector_X_test)
print(classification_report(y_test, pred_3))

              precision    recall  f1-score   support

           0       0.94      0.39      0.55       149
           1       0.80      0.89      0.84       477
           2       0.84      0.90      0.87       489

    accuracy                           0.83      1115
   macro avg       0.86      0.73      0.75      1115
weighted avg       0.84      0.83      0.81      1115



## 7. Klasifikasi sentiment Deep Learning RNN Model : LSTM

In [ ]:
deep_data = pd.read_csv('polarity_dataset.csv')
deep_data.head()

,spell_correction,Sentiment Score,Subjectivity,Overall_Sentiment
0,go jurong point crazy available bugis great world la buffet cine get amore wat,0.2,0.683333,positif
1,ok lar joking wif oni,0.5,0.500000,positif
2,free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate apply,0.4,0.733333,positif
3,dun say early hor already say,0.1,0.300000,positif
4,nah think go usf life around though,0.0,0.000000,netral


In [ ]:
lb=LabelEncoder()
deep_data['Overall_Sentiment'] = lb.fit_transform(deep_data['Overall_Sentiment'])

In [ ]:
tokenizer1 = Tokenizer(num_words=500, split=' ') 
tokenizer1.fit_on_texts(deep_data['spell_correction'].values)

In [ ]:
X = tokenizer1.texts_to_sequences(deep_data['spell_correction'].values)
X = pad_sequences(X)

In [ ]:
METRICS = [tf.keras.metrics.BinaryAccuracy(name = 'accuracy'), 
           tf.keras.metrics.Precision(name = 'precision'), 
           tf.keras.metrics.Recall(name = 'recall'), 
           tf.keras.metrics.FalseNegatives(name = 'False Negative'),
           tf.keras.metrics.FalsePositives(name = 'False Positif'),
           tf.keras.metrics.TruePositives(name = 'True Positif'),
           tf.keras.metrics.TrueNegatives(name = 'True Negative'),
           tf.keras.metrics.MeanSquaredError(name = 'MSE')]

In [ ]:
model = Sequential()
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.add(Flatten())
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = METRICS)
print(model.summary())

Model: "sequential_53"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_40 (Embedding)    (None, 53, 120)           60000     
                                                                 
 spatial_dropout1d_4 (Spatia  (None, 53, 120)          0         
 lDropout1D)                                                     
                                                                 
 lstm_16 (LSTM)              (None, 176)               209088    
                                                                 
 dense_49 (Dense)            (None, 3)                 531       
                                                                 
 flatten_23 (Flatten)        (None, 3)                 0         
                                                                 
Total params: 269,619
Trainable params: 269,619
Non-trainable params: 0
_______________________________________________

In [ ]:
y=pd.get_dummies(deep_data['Overall_Sentiment'])
X_train2, X_test2, y_train2, y_test2 = train_test_split(X,y, test_size = 0.2, random_state = 42)

In [ ]:
batch_size=32
model.fit(X_train2, y_train2, epochs = 10, batch_size=batch_size, verbose = 'auto')

Epoch 1/10
140/140 [==============================] - 46s 297ms/step - loss: 0.8395 - accuracy: 0.7593 - precision: 0.7180 - recall: 0.4575 - False Negative: 2418.0000 - False Positif: 801.0000 - True Positif: 2039.0000 - True Negative: 8113.0000 - MSE: 0.1639
Epoch 2/10
140/140 [==============================] - 36s 254ms/step - loss: 0.4853 - accuracy: 0.8919 - precision: 0.8634 - recall: 0.8026 - False Negative: 880.0000 - False Positif: 566.0000 - True Positif: 3577.0000 - True Negative: 8348.0000 - MSE: 0.0854
Epoch 3/10
140/140 [==============================] - 36s 260ms/step - loss: 0.3754 - accuracy: 0.9203 - precision: 0.8908 - recall: 0.8674 - False Negative: 591.0000 - False Positif: 474.0000 - True Positif: 3866.0000 - True Negative: 8440.0000 - MSE: 0.0645
Epoch 4/10
140/140 [==============================] - 36s 254ms/step - loss: 0.3456 - accuracy: 0.9287 - precision: 0.8996 - recall: 0.8849 - False Negative: 513.0000 - False Positif: 440.0000 - True Positif: 3944.0000 

In [ ]:
model.evaluate(X_test2,y_test2)

35/35 [==============================] - 5s 75ms/step - loss: 0.5229 - accuracy: 0.9007 - precision: 0.8569 - recall: 0.8430 - False Negative: 175.0000 - False Positif: 157.0000 - True Positif: 940.0000 - True Negative: 2073.0000 - MSE: 0.0847


[0.5229068398475647,
 0.9007474184036255,
 0.8568823933601379,
 0.8430493474006653,
 175.0,
 157.0,
 940.0,
 2073.0,
 0.08469263464212418]

In [ ]:
# test predict new data 
new_data_pred = ['sorry i go to beach today']
new_data_pred = tokenizer1.texts_to_sequences(new_data_pred)
new_data_pred = pad_sequences(new_data_pred, maxlen=53, dtype='int32', value=0)
print(new_data_pred)

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0 33  3 27]]


In [ ]:
sentiment = model.predict(new_data_pred,batch_size=4,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

1/1 - 1s - 760ms/epoch - 760ms/step
negative


## 8. Klasifikasi menggunakan Deep Learning RNN Model : Bidirectional 

In [ ]:
model2 = Sequential()
model2.add(Embedding(500, 120, input_length = X.shape[1]))
model2.add(Bidirectional(LSTM(units=176, return_sequences=True)))
model2.add(Bidirectional(LSTM(units=176, return_sequences=False)))
model2.add(Dense(3,activation='sigmoid'))
model2.add(Flatten())
model2.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = METRICS)
print(model2.summary())

Model: "sequential_54"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_41 (Embedding)    (None, 53, 120)           60000     
                                                                 
 bidirectional_8 (Bidirectio  (None, 53, 352)          418176    
 nal)                                                            
                                                                 
 bidirectional_9 (Bidirectio  (None, 352)              744832    
 nal)                                                            
                                                                 
 dense_50 (Dense)            (None, 3)                 1059      
                                                                 
 flatten_24 (Flatten)        (None, 3)                 0         
                                                                 
Total params: 1,224,067
Trainable params: 1,224,067
N

In [ ]:
batch_size=32
model2.fit(X_train2, y_train2, epochs = 10, batch_size=batch_size, verbose = 'auto')

Epoch 1/10
140/140 [==============================] - 103s 649ms/step - loss: 0.7734 - accuracy: 0.7905 - precision: 0.6696 - recall: 0.7335 - False Negative: 1485.0000 - False Positif: 2017.0000 - True Positif: 4087.0000 - True Negative: 9127.0000 - MSE: 0.1414
Epoch 2/10
140/140 [==============================] - 88s 630ms/step - loss: 0.4498 - accuracy: 0.8978 - precision: 0.8821 - recall: 0.8005 - False Negative: 889.0000 - False Positif: 477.0000 - True Positif: 3568.0000 - True Negative: 8437.0000 - MSE: 0.0815
Epoch 3/10
140/140 [==============================] - 88s 632ms/step - loss: 0.3585 - accuracy: 0.9212 - precision: 0.9066 - recall: 0.8515 - False Negative: 662.0000 - False Positif: 391.0000 - True Positif: 3795.0000 - True Negative: 8523.0000 - MSE: 0.0655
Epoch 4/10
140/140 [==============================] - 89s 637ms/step - loss: 0.3349 - accuracy: 0.9258 - precision: 0.9152 - recall: 0.8569 - False Negative: 638.0000 - False Positif: 354.0000 - True Positif: 3819.000

In [ ]:
model2.evaluate(X_test2,y_test2)

35/35 [==============================] - 6s 169ms/step - loss: 0.5948 - accuracy: 0.8706 - precision: 0.8487 - recall: 0.7444 - False Negative: 285.0000 - False Positif: 148.0000 - True Positif: 830.0000 - True Negative: 2082.0000 - MSE: 0.1008


[0.5947661995887756,
 0.8705530762672424,
 0.8486707806587219,
 0.7443946003913879,
 285.0,
 148.0,
 830.0,
 2082.0,
 0.10075654089450836]

## 9. Optimasi Model Bidirectional Dengan Layer Dropout

In [ ]:
model3 = Sequential()
model3.add(Embedding(500, 120, input_length = X.shape[1]))
model3.add(Dropout(rate=0.4))
model3.add(Bidirectional(LSTM(units=176, return_sequences=True)))
model3.add(Dropout(rate=0.4))
model3.add(Bidirectional(LSTM(units=176, return_sequences=False)))
model3.add(Dense(3,activation='sigmoid'))
model3.add(Flatten())
model3.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = METRICS)
print(model3.summary())

Model: "sequential_55"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_42 (Embedding)    (None, 53, 120)           60000     
                                                                 
 dropout_44 (Dropout)        (None, 53, 120)           0         
                                                                 
 bidirectional_10 (Bidirecti  (None, 53, 352)          418176    
 onal)                                                           
                                                                 
 dropout_45 (Dropout)        (None, 53, 352)           0         
                                                                 
 bidirectional_11 (Bidirecti  (None, 352)              744832    
 onal)                                                           
                                                                 
 dense_51 (Dense)            (None, 3)               

In [ ]:
batch_size=32
model3.fit(X_train2, y_train2, epochs = 10, batch_size=batch_size, verbose = 'auto')

Epoch 1/10
140/140 [==============================] - 103s 660ms/step - loss: 0.7873 - accuracy: 0.7846 - precision: 0.6620 - recall: 0.7233 - False Negative: 1542.0000 - False Positif: 2058.0000 - True Positif: 4030.0000 - True Negative: 9086.0000 - MSE: 0.1478
Epoch 2/10
140/140 [==============================] - 91s 648ms/step - loss: 0.5079 - accuracy: 0.8830 - precision: 0.8593 - recall: 0.7759 - False Negative: 999.0000 - False Positif: 566.0000 - True Positif: 3458.0000 - True Negative: 8348.0000 - MSE: 0.0910
Epoch 3/10
140/140 [==============================] - 97s 690ms/step - loss: 0.4188 - accuracy: 0.9106 - precision: 0.8890 - recall: 0.8360 - False Negative: 731.0000 - False Positif: 465.0000 - True Positif: 3726.0000 - True Negative: 8449.0000 - MSE: 0.0728
Epoch 4/10
140/140 [==============================] - 97s 690ms/step - loss: 0.3722 - accuracy: 0.9217 - precision: 0.8989 - recall: 0.8620 - False Negative: 615.0000 - False Positif: 432.0000 - True Positif: 3842.000

In [ ]:
model3.evaluate(X_test2,y_test2)

35/35 [==============================] - 9s 180ms/step - loss: 0.5174 - accuracy: 0.8978 - precision: 0.8643 - recall: 0.8224 - False Negative: 198.0000 - False Positif: 144.0000 - True Positif: 917.0000 - True Negative: 2086.0000 - MSE: 0.0872


[0.517366349697113,
 0.897757887840271,
 0.8642789721488953,
 0.8224215507507324,
 198.0,
 144.0,
 917.0,
 2086.0,
 0.08723127096891403]

## 10. Klasifikasi Menggunakan Deep Learning Model : Baseline Model

In [ ]:
baseline_data = deep_data.copy()
baseline_data.head(2)

,spell_correction,Sentiment Score,Subjectivity,Overall_Sentiment
0,go jurong point crazy available bugis great world la buffet cine get amore wat,0.2,0.683333,2
1,ok lar joking wif oni,0.5,0.500000,2


In [ ]:
lb2=LabelEncoder()
baseline_data['Overall_Sentiment'] = lb2.fit_transform(baseline_data['Overall_Sentiment'])

In [ ]:
NB_WORDS = 500
tokenizer3 = Tokenizer(num_words = NB_WORDS, split = ' ')
tokenizer3.fit_on_texts(baseline_data['spell_correction'].values)
print('Fitted tokenizer on {} documents'.format(tokenizer3.document_count))
print('{} words in dictionary'.format(tokenizer3.num_words))

Fitted tokenizer on 5572 documents
500 words in dictionary


In [ ]:
baseline_seq = tokenizer1.texts_to_sequences(baseline_data['spell_correction'].values)
baseline_seq = pad_sequences(baseline_seq)

In [ ]:
baseline_seq

array([[  0,   0,   0, ..., 225,   1,  68],
       [  0,   0,   0, ...,   8, 234, 341],
       [  0,   0,   0, ...,  26, 212, 275],
       ...,
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,  50, 123,  10],
       [  0,   0,   0, ...,   0, 323, 145]], dtype=int32)

In [ ]:
def one_hot_seq(seqs, nb_features = NB_WORDS):
    ohs = np.zeros((len(seqs), nb_features))
    for i, s in enumerate(seqs):
        ohs[i, s] = 1.
    return ohs

In [ ]:
X_one_hot_enc = one_hot_seq(baseline_seq)

In [ ]:
X_one_hot_enc

array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [ ]:
base_model = Sequential()
base_model.add(layers.Dense(64, activation='relu', input_shape=(NB_WORDS,)))
base_model.add(layers.Dense(64, activation='relu'))
base_model.add(layers.Dense(3, activation='softmax'))
base_model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = METRICS)
print(base_model.summary())

Model: "sequential_56"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_52 (Dense)            (None, 64)                32064     
                                                                 
 dense_53 (Dense)            (None, 64)                4160      
                                                                 
 dense_54 (Dense)            (None, 3)                 195       
                                                                 
Total params: 36,419
Trainable params: 36,419
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_one_hot_enc, y, test_size = 0.2, random_state = 42)

In [ ]:
batch_size = 32
base_model.fit(X_train3, y_train3, epochs=10, verbose = 'auto', batch_size = batch_size)

Epoch 1/10
140/140 [==============================] - 2s 4ms/step - loss: 0.8792 - accuracy: 0.7748 - precision: 0.8261 - recall: 0.4108 - False Negative: 3283.0000 - False Positif: 482.0000 - True Positif: 2289.0000 - True Negative: 10662.0000 - MSE: 0.1561
Epoch 2/10
140/140 [==============================] - 0s 3ms/step - loss: 0.4542 - accuracy: 0.9000 - precision: 0.8736 - recall: 0.8185 - False Negative: 809.0000 - False Positif: 528.0000 - True Positif: 3648.0000 - True Negative: 8386.0000 - MSE: 0.0794
Epoch 3/10
140/140 [==============================] - 0s 3ms/step - loss: 0.3201 - accuracy: 0.9364 - precision: 0.9116 - recall: 0.8959 - False Negative: 464.0000 - False Positif: 387.0000 - True Positif: 3993.0000 - True Negative: 8527.0000 - MSE: 0.0534
Epoch 4/10
140/140 [==============================] - 0s 3ms/step - loss: 0.2624 - accuracy: 0.9453 - precision: 0.9242 - recall: 0.9107 - False Negative: 398.0000 - False Positif: 333.0000 - True Positif: 4059.0000 - True Nega

In [ ]:
base_model.evaluate(X_test3,y_test3)

35/35 [==============================] - 1s 2ms/step - loss: 0.7719 - accuracy: 0.8882 - precision: 0.8377 - recall: 0.8242 - False Negative: 196.0000 - False Positif: 178.0000 - True Positif: 919.0000 - True Negative: 2052.0000 - MSE: 0.0952


[0.7718842625617981,
 0.888191282749176,
 0.8377392888069153,
 0.8242152333259583,
 196.0,
 178.0,
 919.0,
 2052.0,
 0.09522343426942825]

## 10. Klasifikasi Menggunakan Deep Learning Model : Baseline Model Optimasi

In [ ]:
# mengurangi 1 layer dan mengurangi hidden element
base_model_optimasi1 = Sequential()
base_model_optimasi1.add(layers.Dense(32, activation='relu', input_shape=(NB_WORDS,)))
base_model_optimasi1.add(layers.Dense(3, activation='softmax'))
base_model_optimasi1.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = METRICS)
print(base_model_optimasi1.summary())

Model: "sequential_57"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_55 (Dense)            (None, 32)                16032     
                                                                 
 dense_56 (Dense)            (None, 3)                 99        
                                                                 
Total params: 16,131
Trainable params: 16,131
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size = 32
base_model_optimasi1.fit(X_train3, y_train3, epochs=10, verbose = 'auto', batch_size = batch_size)

Epoch 1/10
140/140 [==============================] - 2s 3ms/step - loss: 0.9277 - accuracy: 0.7566 - precision: 0.8462 - recall: 0.3299 - False Negative: 3734.0000 - False Positif: 334.0000 - True Positif: 1838.0000 - True Negative: 10810.0000 - MSE: 0.1660
Epoch 2/10
140/140 [==============================] - 0s 3ms/step - loss: 0.6623 - accuracy: 0.8581 - precision: 0.8507 - recall: 0.6967 - False Negative: 1352.0000 - False Positif: 545.0000 - True Positif: 3105.0000 - True Negative: 8369.0000 - MSE: 0.1222
Epoch 3/10
140/140 [==============================] - 0s 3ms/step - loss: 0.4905 - accuracy: 0.9015 - precision: 0.8844 - recall: 0.8104 - False Negative: 845.0000 - False Positif: 472.0000 - True Positif: 3612.0000 - True Negative: 8442.0000 - MSE: 0.0854
Epoch 4/10
140/140 [==============================] - 0s 3ms/step - loss: 0.4015 - accuracy: 0.9227 - precision: 0.9004 - recall: 0.8638 - False Negative: 607.0000 - False Positif: 426.0000 - True Positif: 3850.0000 - True Neg

In [ ]:
base_model_optimasi1.evaluate(X_test3,y_test3)

35/35 [==============================] - 1s 2ms/step - loss: 0.5305 - accuracy: 0.8993 - precision: 0.8556 - recall: 0.8395 - False Negative: 179.0000 - False Positif: 158.0000 - True Positif: 936.0000 - True Negative: 2072.0000 - MSE: 0.0846


[0.5305178165435791,
 0.8992526531219482,
 0.8555758595466614,
 0.8394618630409241,
 179.0,
 158.0,
 936.0,
 2072.0,
 0.08455228060483932]

In [ ]:
# menambah layer L2 regulasi
base_model_optimasi2 = Sequential()
base_model_optimasi2.add(layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(NB_WORDS,)))
base_model_optimasi2.add(layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
base_model_optimasi2.add(layers.Dense(3, activation='softmax'))
base_model_optimasi2.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = METRICS)
print(base_model_optimasi2.summary())

Model: "sequential_58"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_57 (Dense)            (None, 32)                16032     
                                                                 
 dense_58 (Dense)            (None, 32)                1056      
                                                                 
 dense_59 (Dense)            (None, 3)                 99        
                                                                 
Total params: 17,187
Trainable params: 17,187
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size = 32
base_model_optimasi2.fit(X_train3, y_train3, epochs=10, verbose = 'auto', batch_size = batch_size)

Epoch 1/10
140/140 [==============================] - 2s 3ms/step - loss: 0.9813 - accuracy: 0.7684 - precision: 0.8232 - recall: 0.3885 - False Negative: 3407.0000 - False Positif: 465.0000 - True Positif: 2165.0000 - True Negative: 10679.0000 - MSE: 0.1605
Epoch 2/10
140/140 [==============================] - 0s 3ms/step - loss: 0.6218 - accuracy: 0.8736 - precision: 0.8446 - recall: 0.7608 - False Negative: 1066.0000 - False Positif: 624.0000 - True Positif: 3391.0000 - True Negative: 8290.0000 - MSE: 0.0999
Epoch 3/10
140/140 [==============================] - 0s 4ms/step - loss: 0.4664 - accuracy: 0.9207 - precision: 0.8953 - recall: 0.8631 - False Negative: 610.0000 - False Positif: 450.0000 - True Positif: 3847.0000 - True Negative: 8464.0000 - MSE: 0.0670
Epoch 4/10
140/140 [==============================] - 0s 3ms/step - loss: 0.4085 - accuracy: 0.9309 - precision: 0.9030 - recall: 0.8880 - False Negative: 499.0000 - False Positif: 425.0000 - True Positif: 3958.0000 - True Neg

In [ ]:
base_model_optimasi2.evaluate(X_test3,y_test3)

35/35 [==============================] - 1s 3ms/step - loss: 0.6529 - accuracy: 0.8909 - precision: 0.8415 - recall: 0.8287 - False Negative: 191.0000 - False Positif: 174.0000 - True Positif: 924.0000 - True Negative: 2056.0000 - MSE: 0.0895


[0.6529471278190613,
 0.8908818960189819,
 0.8415300250053406,
 0.82869952917099,
 191.0,
 174.0,
 924.0,
 2056.0,
 0.08954577893018723]

## 11. Klasifikasi Menggunakan Deep Learning Model : Baseline LSTM

In [ ]:
model_lstm_baseline = Sequential()
model_lstm_baseline.add(Embedding(500, 120, input_length = X.shape[1]))
model_lstm_baseline.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
model_lstm_baseline.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model_lstm_baseline.add(Dense(3,activation='softmax'))
model_lstm_baseline.add(Flatten())
model_lstm_baseline.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = METRICS)
print(model_lstm_baseline.summary())

Model: "sequential_59"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_43 (Embedding)    (None, 53, 120)           60000     
                                                                 
 lstm_21 (LSTM)              (None, 53, 176)           209088    
                                                                 
 lstm_22 (LSTM)              (None, 64)                61696     
                                                                 
 dense_60 (Dense)            (None, 3)                 195       
                                                                 
 flatten_26 (Flatten)        (None, 3)                 0         
                                                                 
Total params: 330,979
Trainable params: 330,979
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
batch_size=32
model_lstm_baseline.fit(X_train2, y_train2, epochs = 10, batch_size=batch_size, verbose = 'auto')

Epoch 1/10
140/140 [==============================] - 57s 352ms/step - loss: 0.8171 - accuracy: 0.7948 - precision: 0.7475 - recall: 0.5806 - False Negative: 2337.0000 - False Positif: 1093.0000 - True Positif: 3235.0000 - True Negative: 10051.0000 - MSE: 0.1444
Epoch 2/10
140/140 [==============================] - 50s 354ms/step - loss: 0.4739 - accuracy: 0.8957 - precision: 0.8654 - recall: 0.8136 - False Negative: 831.0000 - False Positif: 564.0000 - True Positif: 3626.0000 - True Negative: 8350.0000 - MSE: 0.0831
Epoch 3/10
140/140 [==============================] - 49s 353ms/step - loss: 0.3640 - accuracy: 0.9252 - precision: 0.8958 - recall: 0.8777 - False Negative: 545.0000 - False Positif: 455.0000 - True Positif: 3912.0000 - True Negative: 8459.0000 - MSE: 0.0617
Epoch 4/10
140/140 [==============================] - 50s 357ms/step - loss: 0.3387 - accuracy: 0.9291 - precision: 0.8999 - recall: 0.8858 - False Negative: 509.0000 - False Positif: 439.0000 - True Positif: 3948.000

In [ ]:
model_lstm_baseline.evaluate(X_test2, y_test2)

35/35 [==============================] - 3s 47ms/step - loss: 0.5609 - accuracy: 0.8969 - precision: 0.8526 - recall: 0.8350 - False Negative: 184.0000 - False Positif: 161.0000 - True Positif: 931.0000 - True Negative: 2069.0000 - MSE: 0.0890


[0.5608978867530823,
 0.8968610167503357,
 0.8525640964508057,
 0.8349775671958923,
 184.0,
 161.0,
 931.0,
 2069.0,
 0.08904077857732773]

## 12. Klasifikasi Menggunakan Deep Learning Model : CCN

In [ ]:
model_cnn = Sequential()
model_cnn.add(Embedding(500, 120, input_length=NB_WORDS))
model_cnn.add(Dropout(0.2))
model_cnn.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model_cnn.add(GlobalMaxPooling1D())
model_cnn.add(Dropout(0.2))
model_cnn.add(Dense(128,activation='relu'))
model_cnn.add(Dense(3,activation='softmax'))
model_cnn.add(Flatten())
model_cnn.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = METRICS)
model_cnn.summary()

Model: "sequential_60"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_44 (Embedding)    (None, 500, 120)          60000     
                                                                 
 dropout_46 (Dropout)        (None, 500, 120)          0         
                                                                 
 conv1d_25 (Conv1D)          (None, 500, 64)           23104     
                                                                 
 global_max_pooling1d_13 (Gl  (None, 64)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_47 (Dropout)        (None, 64)                0         
                                                                 
 dense_61 (Dense)            (None, 128)               8320      
                                                     

In [ ]:
batch_size=32
model_cnn.fit(X_train3, y_train3, epochs = 10, batch_size=batch_size, verbose = 'auto')

Epoch 1/10
140/140 [==============================] - 16s 103ms/step - loss: 0.9998 - accuracy: 0.7121 - precision: 0.7742 - recall: 0.1926 - False Negative: 4499.0000 - False Positif: 313.0000 - True Positif: 1073.0000 - True Negative: 10831.0000 - MSE: 0.1803
Epoch 2/10
140/140 [==============================] - 14s 103ms/step - loss: 0.9852 - accuracy: 0.6735 - precision: 0.5399 - recall: 0.1380 - False Negative: 3842.0000 - False Positif: 524.0000 - True Positif: 615.0000 - True Negative: 8390.0000 - MSE: 0.1995
Epoch 3/10
140/140 [==============================] - 14s 102ms/step - loss: 0.9777 - accuracy: 0.6801 - precision: 0.5503 - recall: 0.2197 - False Negative: 3478.0000 - False Positif: 800.0000 - True Positif: 979.0000 - True Negative: 8114.0000 - MSE: 0.1977
Epoch 4/10
140/140 [==============================] - 14s 102ms/step - loss: 0.9776 - accuracy: 0.6792 - precision: 0.5491 - recall: 0.2096 - False Negative: 3523.0000 - False Positif: 767.0000 - True Positif: 934.0000

In [ ]:
model_cnn.evaluate(X_test3, y_test3)

35/35 [==============================] - 2s 23ms/step - loss: 0.9899 - accuracy: 0.6735 - precision: 0.8108 - recall: 0.0269 - False Negative: 1085.0000 - False Positif: 7.0000 - True Positif: 30.0000 - True Negative: 2223.0000 - MSE: 0.2006


[0.989867091178894,
 0.6735425591468811,
 0.8108108043670654,
 0.026905829086899757,
 1085.0,
 7.0,
 30.0,
 2223.0,
 0.20063547790050507]

## 13. Klasifikasi Menggunakan Deep Learning Model : CNN Optimasi

In [ ]:
model_cnn_optimasi1 = Sequential()
model_cnn_optimasi1.add(Embedding(500, 120, input_length=NB_WORDS))
model_cnn_optimasi1.add(Conv1D(64,kernel_size=3,padding='same',activation='relu',strides=1))
model_cnn_optimasi1.add(MaxPooling1D(pool_size=2))
model_cnn_optimasi1.add(Dropout(0.25))
model_cnn_optimasi1.add(GRU(128,return_sequences=True))
model_cnn_optimasi1.add(Flatten())
model_cnn_optimasi1.add(Dense(128,activation='relu'))
model_cnn_optimasi1.add(Dropout(0.5))
model_cnn_optimasi1.add(Dense(3,activation='softmax'))
model_cnn_optimasi1.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = METRICS)
model_cnn_optimasi1.summary()

Model: "sequential_61"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_45 (Embedding)    (None, 500, 120)          60000     
                                                                 
 conv1d_26 (Conv1D)          (None, 500, 64)           23104     
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 250, 64)          0         
 1D)                                                             
                                                                 
 dropout_48 (Dropout)        (None, 250, 64)           0         
                                                                 
 gru_12 (GRU)                (None, 250, 128)          74496     
                                                                 
 flatten_28 (Flatten)        (None, 32000)             0         
                                                     

In [ ]:
batch_size=32
history = model_cnn_optimasi1.fit(X_train3, y_train3, epochs = 10, batch_size=batch_size, verbose = 'auto')

Epoch 1/10
140/140 [==============================] - 52s 340ms/step - loss: 0.9069 - accuracy: 0.7175 - precision: 0.6592 - recall: 0.3159 - False Negative: 3812.0000 - False Positif: 910.0000 - True Positif: 1760.0000 - True Negative: 10234.0000 - MSE: 0.1837
Epoch 2/10
140/140 [==============================] - 48s 342ms/step - loss: 0.6130 - accuracy: 0.8472 - precision: 0.8040 - recall: 0.7162 - False Negative: 1265.0000 - False Positif: 778.0000 - True Positif: 3192.0000 - True Negative: 8136.0000 - MSE: 0.1128
Epoch 3/10
140/140 [==============================] - 48s 345ms/step - loss: 0.4446 - accuracy: 0.9021 - precision: 0.8697 - recall: 0.8308 - False Negative: 754.0000 - False Positif: 555.0000 - True Positif: 3703.0000 - True Negative: 8359.0000 - MSE: 0.0776
Epoch 4/10
140/140 [==============================] - 49s 347ms/step - loss: 0.3574 - accuracy: 0.9212 - precision: 0.8927 - recall: 0.8678 - False Negative: 589.0000 - False Positif: 465.0000 - True Positif: 3868.000

In [ ]:
model_cnn_optimasi1.evaluate(X_test3, y_test3)

35/35 [==============================] - 4s 83ms/step - loss: 0.6750 - accuracy: 0.8873 - precision: 0.8373 - recall: 0.8215 - False Negative: 199.0000 - False Positif: 178.0000 - True Positif: 916.0000 - True Negative: 2052.0000 - MSE: 0.0914


[0.6750272512435913,
 0.887294352054596,
 0.837294340133667,
 0.8215246796607971,
 199.0,
 178.0,
 916.0,
 2052.0,
 0.09138578921556473]